In [4]:
from thermoclassifier.phases.net import PhaseClassifier
from thermoclassifier.dataset.dataset_creator import *
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
from torch.optim import Adam
import numpy as np
import matplotlib.pyplot as plt

In [5]:
def epoch(net, train_loader, loss_func, optimizer, batch_size, seq_len):
    epoch_losses = np.zeros([len(train_loader), ])
    correct = 0
    incorrect = 0
    
    for i, d in enumerate(train_loader):
        # Scale inputs and get predictions
        #inp = d[:, :, :-1]
        inp = d[:, :, [0, 2]]
        inp[:, :, 0] /= 1000
        predictions = net(inp.float()).squeeze()
        
        # Each batch consists of measurement batches, where seq_len measurements are put into one batch. In such a 
        # measurement batch, every measurement has the same label as it needs to be from the same element an phase. 
        # This leads to a target array where the target class is contained for each measurement in the measurement 
        # batch. With this, CrossEntropyLoss would not work as the predictions are made for the whole measurement 
        # batch and CEL therefore expects only on class label per measurement batch. Therefore, only the first 
        # element of the last dimension of d is considered as target (all the entries in the last dimension are the 
        # same anyways so it could be any entry)
        targets = d[:, :, -1][:, 0].long()
                
        correct += (predictions.argmax(dim=-1) == targets).sum().item()
        incorrect += len(targets) - (predictions.argmax(dim=-1) == targets).sum().item()
        
        #print(predictions.argmax(dim=-1), targets)
        
        # Calculate the loss
        loss = loss_func(predictions, targets)
        epoch_losses[i] = loss
        
        # Backward step
        net.zero_grad()
        loss.backward()
        optimizer.step()
    
    print('Training accuracy: ', correct/(correct + incorrect))
    
    return epoch_losses.mean()

In [9]:
# Training and testing routines

def train(net, train_loader, batch_size, seq_len, save=True):
    # Hyperparameters
    nr_epochs = 500
    lr = 0.01
    
    loss_func = nn.CrossEntropyLoss()
    optimizer = Adam(net.parameters(), lr=lr)
    
    losses = np.zeros([nr_epochs, ])
    
    # Save the net with the lowest training loss as the best net
    best_net = net
    best_loss = epoch(net, train_loader, loss_func, optimizer, batch_size, seq_len)
    
    for i in range(nr_epochs):
        losses[i] = epoch(net, train_loader, loss_func, optimizer, batch_size, seq_len)
        
        if losses[i] < best_loss:
            best_net = net
        
        if i % 10 == 0:
            print(losses[i])
            
    return best_net
        
def test(net, test_loader): 
    correct = 0
    incorrect = 0

    for d in test_loader:
        inp = d[:, :, :-1]
        inp[:, :, 0] /= 1000
        predictions = net(inp.float())
        targets = d[:, :, -1][:, 0].long()
        
        correct += (predictions.argmax(dim=-1).flatten() == targets).sum().item()
        incorrect += len(targets) - (predictions.argmax(dim=-1).flatten() == targets).sum().item()

    accuracy = correct/(correct + incorrect)
    print('Test accuracy: ', accuracy)
    
    return accuracy

In [14]:
# Hyperparameters
seq_len = 1
batch_size = 256

# Create the dataset
measurement = 'C'
dc = DatasetCreator(elements=None, splits=(0.8, 0.2), validation=False, seq_len=seq_len, measurement=measurement, user='phase', step=0.05)
train_dataset, test_dataset, val_dataset = dc.get_datasets()

# Create the DataLoaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
if val_dataset:
    val_loader = DataLoader(val_dataset)

KeyboardInterrupt: 

In [ ]:
# Create the network
t = True
net = PhaseClassifier(train=t, measurement=measurement)

In [ ]:
# Train the network
best_net = train(net, train_loader, batch_size, seq_len)

# Test the trained network
test(best_net, test_loader)

In [43]:
net = torch.load(r"C:\Users\danie\Documents\Montanuni\Masterarbeit\5 Programmcodes\packages\ThermoClassifier\ThermoClassifier\phases\models\new\PhaseClassifier.pth")
test(net, test_loader)

22862 2896
Test accuracy:  0.8875689106297073


0.8875689106297073

In [55]:
#torch.save(best_net, 'PhaseClassifier_8778.pth')